In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

In [1]:
from model import *

import tensorflow as tf
import numpy as np
import os
import pickle

from sklearn.model_selection import train_test_split


update_config = {
    'seed' : 0,
    'num_classes' : 100,
    'local_epochs' : 10,
    'local_batch_size' : 100,
}

client_config = {
    'num' : 100,
}

class Client():
    def __init__(self, client_id : int, update_config : dict, weights):
        self.client_id = client_id
        self.update_config = update_config
        # self.trainData = self.unpickle(os.path.join('/home/joongho/FL/', 'data/clients/client' + str(self.client_id)))
        self.trainData = self.unpickle(os.path.join(os.path.dirname(__file__), 'data/clients/client' + str(self.client_id)))
        self.pre_weights = weights

    def unpickle(self, file):
        with open(file, 'rb') as fo:
            myDict = pickle.load(fo, encoding='latin1')
        return myDict

    def update_model(self):
        self.MobileNet = MobileNet_RS([self.trainData['data'].shape[2], self.trainData['data'].shape[3],
                                    self.trainData['data'].shape[1]], 
                                    num_classes = self.update_config['num_classes'])
        # self.MobileNet = MobileNet([self.trainData['data'].shape[2], self.trainData['data'].shape[3],
        #                     self.trainData['data'].shape[1]], 
        #                     num_classes = self.update_config['num_classes'])
        self.model = self.MobileNet.model

        # self.model.set_weights(self.pre_weights)
        x = np.transpose(self.trainData['data'], (0,2,3,1))
        x_n = x / 255
        y = np.eye(self.update_config['num_classes'])[self.trainData['fine_labels']]

        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(x_n, y, test_size = 0.1)
        self.model.fit(
            x = self.x_train,
            y = self.y_train,
            batch_size = self.update_config['local_batch_size'],
            epochs = self.update_config['local_epochs'],
            validation_data = [self.x_val, self.y_val],
            verbose = 0,
        )
        self.weights = self.model.get_weights()
        self.weights = np.array(self.weights)

    def diff_weights(self):
        self.update_model()
        return self.pre_weights - self.weights
    
    def send_update(self):
        return self.diff_weights()
        

In [3]:
client = Client(client_id = 1, update_config = update_config, server_config = server_config)

In [4]:
client.update_model()

Epoch 1/10
135/135 [==============================] - 12s 37ms/step - loss: 3.4198 - acc: 0.0844 - val_loss: 4.3514 - val_acc: 0.0387
Epoch 2/10
135/135 [==============================] - 4s 30ms/step - loss: 2.9189 - acc: 0.1700 - val_loss: 3.8835 - val_acc: 0.0347
Epoch 3/10
135/135 [==============================] - 4s 33ms/step - loss: 2.6431 - acc: 0.2369 - val_loss: 3.6130 - val_acc: 0.0347
Epoch 4/10
135/135 [==============================] - 4s 32ms/step - loss: 2.4585 - acc: 0.2865 - val_loss: 3.5320 - val_acc: 0.0313
Epoch 5/10
135/135 [==============================] - 4s 33ms/step - loss: 2.3071 - acc: 0.3265 - val_loss: 3.5527 - val_acc: 0.0347
Epoch 6/10
135/135 [==============================] - 4s 33ms/step - loss: 2.1975 - acc: 0.3539 - val_loss: 3.5554 - val_acc: 0.0340
Epoch 7/10
135/135 [==============================] - 5s 38ms/step - loss: 2.1103 - acc: 0.3803 - val_loss: 3.6227 - val_acc: 0.0340
Epoch 8/10
135/135 [==============================] - 5s 39ms/step -

/tmp/ipykernel_1281254/1744656522.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.weights = np.array(self.weights)
